In [ ]:
from collections import Counter
from pathlib import Path

from IPython.display import display, Markdown
import matplotlib.pyplot
import netCDF4
import numpy as np
import numexpr
import pandas as pd
import seaborn
import xarray as xr

import plot

In [ ]:
def index_over_dim(array, reduce_dim, index_array):
    assert len(array.shape) == len(index_array.shape) + 1
    
    non_reduced_indices = list(np.mgrid[tuple(range(0, i) for i in index_array.shape)])
    indices = [None] * len(array.shape)
    for i, dim in enumerate(array.dims):
        if dim == reduce_dim:
            indices[i] = index_array.values
        else:
            index_array.dims.index(dim)
            indices[i] = non_reduced_indices[index_array.dims.index(dim)]
    indices = tuple(indices)
    
    dims = tuple(d for d in array.dims if d != reduce_dim)
    coords = {n: v for n, v in array.coords.items() if reduce_dim not in v.dims}
    
    return xr.DataArray(
        array.values[indices],
        dims=dims,
        coords=coords,
    )

In [ ]:
data_ds = xr.open_dataset('../data/processed/concat-data.nc')

## Coarse Grid Search ##

In [ ]:
bic_coarse_pth = Path("../models/sfa_mri_cad/parameter_sweep_coarse-bics.nc")

In [ ]:
bics_cs_ds = xr.open_dataset(str(bic_coarse_pth)).load()
display(bics_cs_ds)

In [ ]:
def bics_to_array(bics_ds):
    bics_sel = bics_ds.sel(model=bics_ds['n_iter'] < 9998)
    bics_nona = bics_sel.sel(model=bics_sel.to_array().isnull().sum('variable') == 0)
    bics_array = bics_nona.set_index(model=['l_gexp', 'l_mri', 'alpha', 'k']).unstack('model')
    bics_array.attrs['empty_model_bic'] = bics_ds.attrs['empty_model_bic']
    
    return bics_array

bics_cs_array = bics_to_array(bics_cs_ds)
display(bics_cs_array)

Best BIC over all k and alpha, for lambda penalties of the datatypes.

In [ ]:
def heatmap_l_penalty_bic(bics_array):
    plot.heatmap(
        bics_array['bic'].fillna(bics_array['bic'].max()).min(['alpha', 'k']),
        zlim=[float(bics_cs_array['bic'].min()), bics_array.attrs['empty_model_bic']],
        zlabel='BIC',
    )
heatmap_l_penalty_bic(bics_cs_array)

In [ ]:
def heatmaps_l_penalty_over_k(bics_array):
    for k in bics_array['k'].values:
        display(Markdown(f"### k={k} ###"))
        ba = bics_array.sel(k=k).drop('k')
        alpha_idx = ba['bic'].fillna(ba['bic'].max()).argmin('alpha')
        plot.heatmap(
            index_over_dim(ba['bic'], 'alpha', alpha_idx),
            zlim=[float(ba['bic'].min()), bics_array.attrs['empty_model_bic']],
            zlabel='BIC',
        )
        plot.heatmap(
            xr.DataArray(ba.coords['alpha'][np.asarray(alpha_idx)].values, alpha_idx.coords),
            cmap='inferno',
            zlabel='alpha'
        )
        plot.heatmap(
            index_over_dim(ba['deviance_gexp'], 'alpha', alpha_idx),
            zlabel='Deviance (gexp)',
        )
        plot.heatmap(
            index_over_dim(ba['deviance_mri'], 'alpha', alpha_idx),
            zlabel='Deviance (MRI)',
        )
        plot.heatmap(
            index_over_dim(ba['dof_gexp'], 'alpha', alpha_idx),
            zlabel='Degrees of Freedom (gexp)',
        )
        plot.heatmap(
            index_over_dim(ba['dof_mri'], 'alpha', alpha_idx),
            zlabel='Degrees of Freedom (MRI)',
        )
        plot.heatmap(
            index_over_dim(ba['sparsity_gexp'], 'alpha', alpha_idx),
            zlim=[0, 1], cmap='Greys',
            zlabel='Sparsity (gexp)',
        )
        plot.heatmap(
            index_over_dim(ba['sparsity_mri'], 'alpha', alpha_idx),
            zlim=[0, 1], cmap='Greys',
            zlabel='Sparsity (MRI)',
        )
        
heatmaps_l_penalty_over_k(bics_cs_array)

In [ ]:
with plot.subplots(1, 1, figsize=(10, 3)) as (fig, ax):
    plot.lines(
        bics_cs_array['bic'].min(['alpha', 'l_gexp', 'l_mri']),
        ax=ax,
    )

## Finer Grid Search ##

In [ ]:
bic_fine_pth = Path("../models/sfa_mri_cad/parameter_sweep_fine-bics.nc")

In [ ]:
bics_fs_ds = xr.open_dataset(str(bic_fine_pth)).load()

In [ ]:
bics_fs_array = bics_to_array(bics_fs_ds)
display(bics_fs_array)

In [ ]:
heatmap_l_penalty_bic(bics_fs_array)

In [ ]:
heatmaps_l_penalty_over_k(bics_fs_array)

In [ ]:
bics_fs_array

In [ ]:
display(bics_fs_array['bic'].min())
min_idx = tuple(i[0] for i in np.where(bics_fs_array['bic'].min() == bics_fs_array['bic']))
bics_fs_array['bic'][min_idx]

In [ ]:
bics_sel = bics_fs_ds.sel(model=bics_fs_ds['n_iter'] < bics_fs_ds['n_iter'].max())
bics_sel.isel(model=int(bics_sel['bic'].argmin()))

In [ ]:
bics_sel